# Indexing Reward Unit Test
Performing verification of the indexing rewards mechanism by comparing the event data to the results of cadCAD model.

In [1]:
import pandas as pd
import numpy as np
from decimal import Decimal
from decimal import *
import model.parts.utils as utils
from model.sys_params import *
df = pd.read_pickle(r'experiment.p')

stakeDelegateds: 605 events
stakeDelegatedLockeds: 375 events
stakeDelegatedWithdrawns: 326 events
allocationCloseds: 47 events
allocationCollecteds: 16 events
stakeDepositeds: 25 events
rewardsAssigneds: 47 events
delegationParametersUpdateds: 6 events
allocationCreateds: 51 events
TOTAL NUMBER OF EVENTS: 1529
You should set SIMULATION_TIME_STEPS in config.py to a minimum of 1529 to capture all events.



Configure environment

In [2]:
df.reset_index(inplace = True)
pd.set_option('display.max_rows', None)


In [3]:

print("EXPECTED TRUE INDEXING REWARDS FROM REWARDS ASSIGNED EVENTS:")
is_first = True
for timestep, indexing_fee_events_list in rewards_assigned_events.items():
    # if is_first:
    #     is_first = False
    #     pass
    # else:        
        if indexing_fee_events_list is None:
            indexing_fee_amt = 0
        else:
            indexing_fee_amt = sum([e['amount'] for e in indexing_fee_events_list])        
        print(indexing_fee_amt)
        # print(f"{timestep}, {stake_deposited_event[0]['tokens']}")
    


EXPECTED TRUE INDEXING REWARDS FROM REWARDS ASSIGNED EVENTS:
79000.322109085104702444
78681.945634908142661532
37810.500232191486929607
26381.732043325393053999
25494.839303727812415387
26234.529693224212529763
49891.90274540201605701
49891.817848670854079448
66214.393501266041029992
31254.0437297414733718
98402.206621725694544502
217207.865614270991850845
265654.897596226056278932
213941.342001342071645712
194137.522791957485390532
91859.973399058188810578
135410.384627951762628518
237821.675242826595473
173947.355266842636556792
303845.448615810697843104
151930.725822086935104
2792.339036562857078894
22918.683845921654696336
23950.058834745539489506
31780.514450169181908583
32679.131508368940869594
12373.914467045118773128
10408.072875288181882076
10238.437833754494711791
10933.286787831162710644
17868.796741382750311832
49939.430091121812419
73035.525482052350460548
37547.80161290838608336
60965.34970625528535342
70875.47292936929357896
74148.542616902969932048
72239.677999385421190

In [4]:
print("MODELED RESULTS")
# TODO: weave in indexer_revenue_cut events
# indexer #1
# indexer_revenue_cut = 0.89
# indexer #2
indexer_revenue_cut = Decimal(0.8)
# print(df.iloc[timestep])
# is_first = True
indexing_rewards_modeled = {}
for timestep, stake_deposited_event in rewards_assigned_events.items():
    # back indexing rewards out from increase in pool_delegated_stake / 0.11 * 0.89
    event = stake_deposited_event[0]
    new_indexing_rewards = df.iloc[timestep-1].indexers[event['indexer']].pool_delegated_stake
    old_indexing_rewards = df.iloc[timestep-2].indexers[event['indexer']].pool_delegated_stake
    indexing_rewards =(new_indexing_rewards - old_indexing_rewards) / (1 - indexer_revenue_cut) 
    indexing_rewards_modeled[timestep] = indexing_rewards
    print(f"{timestep}, {indexing_rewards=}")
        


MODELED RESULTS
83, indexing_rewards=Decimal('79000.32210908512224403931166')
86, indexing_rewards=Decimal('78681.94563490816013243353323')
164, indexing_rewards=Decimal('37810.50023219149532522458607')
168, indexing_rewards=Decimal('26381.73204332539891192026880')
171, indexing_rewards=Decimal('25494.83930372781807637852082')
174, indexing_rewards=Decimal('26234.52969322421835499878113')
294, indexing_rewards=Decimal('49891.90274540202713523783406')
297, indexing_rewards=Decimal('49891.81784867086515765698320')
373, indexing_rewards=Decimal('66214.39350126605573254084534')
406, indexing_rewards=Decimal('31254.04372974148031159179228')
463, indexing_rewards=Decimal('98402.20662172571639418109307')
544, indexing_rewards=Decimal('217207.8656142710400806797069')
634, indexing_rewards=Decimal('265654.8975962261152661687831')
776, indexing_rewards=Decimal('213941.3420013421191502327618')
867, indexing_rewards=Decimal('194137.5227919575284977215495')
946, indexing_rewards=Decimal('91859.9733

In [5]:
print("UNITTEST RESULTS")
is_first = True
# print(rewards_assigned_events)
cntExact = 0
cntReallyClose = 0
cntClose = 0
cnt = 0
cntWrong = 0
for timestep, rewards_assigned_event in rewards_assigned_events.items():
    # back indexing rewards out from increase in pool_delegated_stake / 0.11 * 0.89
    modeled_indexing_rewards = indexing_rewards_modeled[timestep]
    event_indexing_rewards = rewards_assigned_event[0]['amount']
    print(f"Event: Timestep={timestep}, Indexing Reward Tokens={event_indexing_rewards}")
    print(f"Model: Timestep={timestep}, Indexing Reward Tokens={modeled_indexing_rewards}")
    # print("Tokens Within 1%?", 0.99 <= abs(event_indexing_rewards / modeled_indexing_rewards) <= 1.01)
    # print()
    try:
        ratio = abs(modeled_indexing_rewards / event_indexing_rewards)
    except:
        ratio = np.inf
    exact = ratio == 1.0
    close = 0.99 <= ratio <= 1.01
    reallyClose = 0.99999 <= ratio <= 1.00001

    if exact:            
        cntExact += 1
    elif reallyClose:
        cntReallyClose += 1
    elif close:
        cntClose += 1
    else:
        cntWrong += 1
    cnt += 1
    print("Tokens Equal?", exact)
    print("Tokens Within 0.001%?", reallyClose)
    print("Tokens Within 1%?", close)
    print(f"Exact: {cntExact}, ReallyClose: {cntReallyClose}, Close: {cntClose}, Wrong: {cntWrong}, Total Number: {cnt}")   
    print()
print(f"Exact: {cntExact}, ReallyClose: {cntReallyClose}, Close: {cntClose}, Wrong: {cntWrong}, Total Number: {cnt}")           


UNITTEST RESULTS
Event: Timestep=83, Indexing Reward Tokens=79000.322109085104702444
Model: Timestep=83, Indexing Reward Tokens=79000.32210908512224403931166
Tokens Equal? False
Tokens Within 0.001%? True
Tokens Within 1%? True
Exact: 0, ReallyClose: 1, Close: 0, Wrong: 0, Total Number: 1

Event: Timestep=86, Indexing Reward Tokens=78681.945634908142661532
Model: Timestep=86, Indexing Reward Tokens=78681.94563490816013243353323
Tokens Equal? False
Tokens Within 0.001%? True
Tokens Within 1%? True
Exact: 0, ReallyClose: 2, Close: 0, Wrong: 0, Total Number: 2

Event: Timestep=164, Indexing Reward Tokens=37810.500232191486929607
Model: Timestep=164, Indexing Reward Tokens=37810.50023219149532522458607
Tokens Equal? False
Tokens Within 0.001%? True
Tokens Within 1%? True
Exact: 0, ReallyClose: 3, Close: 0, Wrong: 0, Total Number: 3

Event: Timestep=168, Indexing Reward Tokens=26381.732043325393053999
Model: Timestep=168, Indexing Reward Tokens=26381.73204332539891192026880
Tokens Equal? Fa

# Conclusion
The indexing reward token quantities are accurate to roughly 21 significant figures except in timestep 58.  